## PySpark Aggregation

PySpark `groupBy()` function is used to collect the identical data into groups on DataFrame and perform aggregate functions on the grouped data.

| func | desc |
| - | -|
|count() | Returns the count of rows for each group. |
|mean()  | Returns the mean of values for each group. |
|max()   | Returns the maximum of values for each group. |
|min()   | Returns the minimum of values for each group. |
|sum()   | Returns the total for values for each group. |
|avg()   | Returns the average for values for each group. |
|agg()   | Using this function, we can calculate more than one aggregate at a time. |
|pivot() | This function is used to Pivot the DataFrame (see pivoting notebook) |

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark Aggregation').getOrCreate()

In [0]:
data = [
    ('Sam', 'Software Engineer', 'US', 5000, 30, 500),
    ('Adam', 'Data Scientist', 'US', 6000, 58, 550),
    ('Jonas', 'Sales Person', 'Wales', 5000, 41, 500),
    ('Peter', 'CTO', 'Ireland', 10000, 35, 1500),
    ('Ann', 'Data Analyst', 'Australia', 6000, 24, 500),
    ('Ralph', 'CEO', 'Germany', 15000, 50, 2500),
    ('Lekhana', 'Advertising', 'England', 4500, 27, 560),
    ('Tomas', 'Marketing', 'Hungary', 4500, 30, 570),
    ('Nick', 'Data Engineer', 'Ireland', 5000, 41, 600),
    ('Wade', 'Data Engineer', 'Scotland', 5500, 25, 600)
]

columns = ['name', 'job', 'country', 'salary', 'age', 'bonus']

df = spark.createDataFrame(data = data, schema = columns)

df.printSchema()
df.show(truncate=False)

root
-- name: string (nullable = true)
-- job: string (nullable = true)
-- country: string (nullable = true)
-- salary: long (nullable = true)
-- age: long (nullable = true)
-- bonus: long (nullable = true)

+-------+-----------------+---------+------+---+-----+
name |job |country |salary|age|bonus|
+-------+-----------------+---------+------+---+-----+
Sam |Software Engineer|US |5000 |30 |500 |
Adam |Data Scientist |US |6000 |58 |550 |
Jonas |Sales Person |Wales |5000 |41 |500 |
Peter |CTO |Ireland |10000 |35 |1500 |
Ann |Data Analyst |Australia|6000 |24 |500 |
Ralph |CEO |Germany |15000 |50 |2500 |
Lekhana|Advertising |England |4500 |27 |560 |
Tomas |Marketing |Hungary |4500 |30 |570 |
Nick |Data Engineer |Ireland |5000 |41 |600 |
Wade |Data Engineer |Scotland |5500 |25 |600 |
+-------+-----------------+---------+------+---+-----+

#### groupBy and aggregate on DataFrame columns

In [0]:
df.groupBy('job').avg('salary').show(truncate=False)

+-----------------+-----------+
job |avg(salary)|
+-----------------+-----------+
Software Engineer|5000.0 |
Data Scientist |6000.0 |
Sales Person |5000.0 |
CTO |10000.0 |
Data Analyst |6000.0 |
CEO |15000.0 |
Advertising |4500.0 |
Marketing |4500.0 |
Data Engineer |5250.0 |
+-----------------+-----------+

In [0]:
df.groupBy('job').count().show()

+-----------------+-----+
 job|count|
+-----------------+-----+
Software Engineer| 1|
 Data Scientist| 1|
 Sales Person| 1|
 CTO| 1|
 Data Analyst| 1|
 CEO| 1|
 Advertising| 1|
 Marketing| 1|
 Data Engineer| 2|
+-----------------+-----+

In [0]:
df.groupBy('job').min('salary').show()

+-----------------+-----------+
 job|min(salary)|
+-----------------+-----------+
Software Engineer| 5000|
 Data Scientist| 6000|
 Sales Person| 5000|
 CTO| 10000|
 Data Analyst| 6000|
 CEO| 15000|
 Advertising| 4500|
 Marketing| 4500|
 Data Engineer| 5000|
+-----------------+-----------+

In [0]:
df.count()

Out[7]: 10

#### Aggregate on multiple columns

In [0]:
df.groupBy('country').sum('salary','bonus').show()

+---------+-----------+----------+
 country|sum(salary)|sum(bonus)|
+---------+-----------+----------+
 US| 11000| 1050|
 Wales| 5000| 500|
 Ireland| 15000| 2100|
Australia| 6000| 500|
 Germany| 15000| 2500|
 England| 4500| 560|
 Hungary| 4500| 570|
 Scotland| 5500| 600|
+---------+-----------+----------+

#### Running more aggregates at a time

In [0]:
df \
.groupBy('country') \
.agg( \
  sum('salary').alias('sum_salary'), \
  avg('salary').alias('avg_salary'), \
  sum('bonus').alias('sum_bonus'), \
  max('bonus').alias('max_bonus') \
) \
.show()

+---------+----------+----------+---------+---------+
 country|sum_salary|avg_salary|sum_bonus|max_bonus|
+---------+----------+----------+---------+---------+
 US| 11000| 5500.0| 1050| 550|
 Wales| 5000| 5000.0| 500| 500|
 Ireland| 15000| 7500.0| 2100| 1500|
Australia| 6000| 6000.0| 500| 500|
 Germany| 15000| 15000.0| 2500| 2500|
 England| 4500| 4500.0| 560| 560|
 Hungary| 4500| 4500.0| 570| 570|
 Scotland| 5500| 5500.0| 600| 600|
+---------+----------+----------+---------+---------+

#### The end of the notebook